Imports

In [11]:
from PIL import Image
import numpy as np
from pylfsr import LFSR

## Helper Functions

- Any type to Binary Converter


In [12]:
def convertToBinary(data):
    if isinstance(data, str):
        return ''.join([ format(ord(i), "08b") for i in data ])
    elif isinstance(data, bytes) or isinstance(data, np.ndarray):
        return [ format(i, "08b") for i in data ]
    elif isinstance(data, int) or isinstance(data, np.uint8):
        return format(data, "08b")
    else:
        raise TypeError("Type not supported.")

- Binary to Decimal Converter

In [13]:
def binaryToDecimal(binary):
    decimal = 0
    for bit in binary:
        decimal = decimal*2 + int(bit)
    return decimal

- Convert image to matrix

In [14]:
def getImageMatrix(imageName):

    imageHandler = Image.open(imageName)
    pixels = imageHandler.load()
    color = 1
    if type(pixels[0,0]) == int:
        color = 0
    image_size = imageHandler.size
    
    rows = int(image_size[0])
    cols = int(image_size[1])
    image_matrix = []
    for row in range(rows):
        current_row = []
        for col in range(cols):
            current_row.append(pixels[row,col])
        image_matrix.append(current_row)
    
    return image_matrix, image_size, color


## 2D Chaotic Map

- Hennon Map (Binary)

In [15]:
def HennonMapBinary(dimension, key):

    rows = dimension[0]
    cols = dimension[1]
    x = key[0]
    y = key[1]
    a = 1.4
    b = 0.3

    # Total Number of bitSequence produced
    sequenceSize = rows * cols
    bitSequenceSize = cols
    # Each bitSequence contains k bits
    bitSequence = []       
    # Each matrix contains m*n byteArray
    matrix = []

    for i in range(sequenceSize):
        x_next = y + 1 - (a * (x**2)) 
        y_next = b * x
        x = x_next
        y = y_next
        if x <= 0.4:
            bit = 0
        else:
            bit = 1

        bitSequence.append(bit) 

        # If Bit Sequence has k bits, convert it to decimal and add it to Byte Sequence.
        if i % bitSequenceSize == bitSequenceSize - 1:
            matrix.append(bitSequence)
            bitSequence = []

    return matrix

## Encoding


In [16]:
def encode(imageName, filename, key, poly):

    # read the image.
    imagehandler = Image.open(imageName) 
    image = imagehandler.load()
    image_size = imagehandler.size

    # Get secret data.
    filehandler = open(filename,"r")
    secret_data = "".join(filehandler.readlines())
    secret_data += "=!"
    filehandler.close()

    # convert data to binary.
    binary_secret_data = convertToBinary(secret_data)
    data_len = len(binary_secret_data)

    # Create and load intermediate images.
    intermediate_imagehandler1 = Image.new(mode="RGB", size=image_size)
    intermediate_image1 = intermediate_imagehandler1.load()
    intermediate_imagehandler2 = Image.new(mode="RGB", size=image_size)
    intermediate_image2 = intermediate_imagehandler2.load()

    # Build chaotic map and derive some available pixels.
    map = HennonMapBinary(image_size, key)

    # Select pixels by superimposing Chaotic Map on the image.
    availablePixels = []
    for i in range(image_size[0]):
        for j in range(0,image_size[1]):
            if map[i][j] == 1:
                availablePixels.append([i,j,0])
                intermediate_image1[i,j] = image[i,j]

    # Save intermediate 1 with selected pixels by Chaotic Map.
    intermediate_imagehandler1.save("outputs/image_steg/intermediate1.png","PNG")

    # check if encoding is possible
    if data_len > len(availablePixels)*6:
        raise ValueError("[!] Insufficient space. Please change key or data.")

    # Build LFSR
    lfsr = LFSR(initstate = [1, 1, 1, 1, 1, 1, 1, 1], fpoly=poly, counter_start_zero=True)

    index = 0

    for i in range(0, data_len, 2):

        # Choose pixel using LFSR from Available pixels
        index = (index + binaryToDecimal(lfsr.state)) % len(availablePixels)
        lfsr.next()

        # If pixel is already used 3 times, keep choosing next pixel until unused pixel is found.
        count = 0
        while availablePixels[index][2] >= 2:
            index = (index + binaryToDecimal(lfsr.state)) % len(availablePixels)
            lfsr.next()
            count += 1
            if count > 100:
                print("exceeded")
                return

        '''
        EACH PIXEL CAN BE USED AT MAX 3 TIMES. WE KEEP COUNT OF NUMBER OF USES AS USAGESTATE.

        IF USAGESTATE IS 0: XOR 2 DATA BITS WITH 2 LSB OF RED VALUE
        IF USAGESTATE IS 1: XOR 2 DATA BITS WITH 2 LSB OF BLUE VALUE
        IF USAGESTATE IS 2: XOR 2 DATA BITS WITH 2 LSB OF GREEN VALUE
        '''

        # Extract row, column and usagestate values.
        row, col, usageState = availablePixels[index][0], availablePixels[index][1], availablePixels[index][2]
        
        # Select pixel in intermediate 2.
        intermediate_image2[row,col] = image[row,col]

        # Update with Red Value
        if usageState == 0:
            red = list(convertToBinary(image[row,col][0]))
            red[-2] = str(int(red[-2]) ^ int(binary_secret_data[i]))
            red[-1] = str(int(red[-1]) ^ int(binary_secret_data[i+1]))
            red = "".join(red)
            red = binaryToDecimal(red)
            image[row,col] = (red, image[row,col][1], image[row,col][2])

        # Update with Blue Value
        elif usageState == 1:
            blue = list(convertToBinary(image[row,col][1]))
            blue[-2] = str(int(blue[-2]) ^ int(binary_secret_data[i]))
            blue[-1] = str(int(blue[-1]) ^ int(binary_secret_data[i+1]))
            blue = "".join(blue)
            blue = binaryToDecimal(blue)
            image[row,col] = (image[row,col][0], blue, image[row,col][2])
        
        # Update with Green Value
        elif usageState == 2:
            green = list(convertToBinary(image[row,col][2]))
            green[-2] = str(int(green[-2]) ^ int(binary_secret_data[i]))
            green[-1] = str(int(green[-1]) ^ int(binary_secret_data[i+1]))
            green = "".join(green)
            green = binaryToDecimal(green)
            image[row,col] = (image[row,col][0], image[row,col][1], green)
        
        availablePixels[index][2] += 1

    # Save imtermediate 2 with total Used Bits after encoding with LFSR select
    intermediate_imagehandler2.save("outputs/image_steg/intermediate2.png","PNG")

    # Save the final image.
    imagehandler.save("outputs/image_steg/encoded_image.png", "PNG")

In [17]:
import json

def main():
    info = open("config.json","r")
    json_data = json.load(info)
    original_image = json_data["source"]["image_filename"]
    hennon_key = json_data["key"]["hennon_key"]
    lfsr_poly = json_data["key"]["lfsr_polynomial"]
    
    encode(original_image, "assets/input1000.txt", hennon_key, lfsr_poly)

In [18]:
main()